<a href="https://colab.research.google.com/github/Aasthapaudel/-Imagine-cup-cloud-skill-Challenge-/blob/main/working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
pip install -U langchain-community datasets

In [7]:
from datasets import load_dataset

dataset_name="teksingh/NagarGPT-dataset0"

In [6]:
# First install required packages

!pip install transformers datasets faiss-cpu sentence-transformers langchain chromadb


import torch

from transformers import AutoTokenizer, AutoModelForCausalLM

from sentence_transformers import SentenceTransformer

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceEmbeddings

import textwrap



def load_hf_dataset(dataset_name):

  data=load_dataset(dataset_name,split="train")

  output_texts = []

  for i in range(len(data['मानव'])):

      text = f"### Question: {data['मानव'][i]}\n ### Answer: {data['सहायक'][i]}"

      output_texts.append(text)

  return output_texts



# class RAGSystem:

#     def __init__(self, llm_model="bigscience/bloom-7b1",

#                  embedding_model="sentence-transformers/all-mpnet-base-v2"):

#         # Initialize the LLM

#         self.tokenizer = AutoTokenizer.from_pretrained(llm_model)

#         self.model = AutoModelForCausalLM.from_pretrained(

#             llm_model,

#             device_map="auto",

#             torch_dtype=torch.float16

#         )



#         # Initialize embedding model

#         self.embeddings = HuggingFaceEmbeddings(

#             model_name=embedding_model,

#             model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}

#         )



#         # Initialize text splitter

#         self.text_splitter = RecursiveCharacterTextSplitter(

#             chunk_size=1000,

#             chunk_overlap=200

#         )



#         self.vector_store = None



#     def add_documents(self, documents):

#         """

#         Add documents to the vector store for retrieval

#         documents: list of strings

#         """

#         # Split documents into chunks

#         splits = self.text_splitter.create_documents(documents)



#         # Create or update vector store

#         self.vector_store = Chroma.from_documents(

#             documents=splits,

#             embedding=self.embeddings

#         )



#     def generate_response(self, query, k=3, max_length=512):

#         """

#         Generate a response using RAG

#         query: string containing the question

#         k: number of relevant documents to retrieve

#         max_length: maximum length of generated response

#         """

#         # Retrieve relevant documents

#         if self.vector_store is None:

#             raise ValueError("No documents added to vector store yet!")



#         retrieved_docs = self.vector_store.similarity_search(query, k=3)



#         #Construct prompt with retrieved context

#         context = "\n".join([doc.page_content for doc in retrieved_docs])

#         prompt = f"""Context: {context}\n\nQuestion: {query}\n\nAnswer:"""



#         # Generate response

#         inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

#         outputs = self.model.generate(

#             **inputs,

#             max_length=max_length,

#             num_beams=4,

#             temperature=0.7,

#             top_p=0.9,
#             do_sample=True,

#         )



#         response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

#         return response
#         return retrieved_docs


# # Example usage


class RAGSystem:

    def __init__(self, llm_model="bigscience/bloom-7b1", embedding_model="sentence-transformers/all-mpnet-base-v2"):
        # Initialize the LLM
        self.tokenizer = AutoTokenizer.from_pretrained(llm_model)
        self.model = AutoModelForCausalLM.from_pretrained(
            llm_model,
            device_map="auto",
            torch_dtype=torch.float16
        )

        # Initialize embedding model
        self.embeddings = HuggingFaceEmbeddings(
            model_name=embedding_model,
            model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
        )

        # Initialize text splitter
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200
        )

        self.vector_store = None

    def add_documents(self, documents):
        """
        Add documents to the vector store for retrieval
        documents: list of strings
        """
        # Split documents into chunks
        splits = self.text_splitter.create_documents(documents)

        # Create or update vector store
        self.vector_store = Chroma.from_documents(
            documents=splits,
            embedding=self.embeddings
        )

    def generate_response(self, query, k=3, max_length=512):
        """
        Generate a response using RAG
        query: string containing the question
        k: number of relevant documents to retrieve
        max_length: maximum length of generated response
        """
        # Retrieve relevant documents
        if self.vector_store is None:
            raise ValueError("No documents added to vector store yet!")

        # Retrieve the top 'k' relevant documents
        retrieved_docs = self.vector_store.similarity_search(query, k=k)

        # Construct prompt with retrieved context
        context = "\n".join([doc.page_content for doc in retrieved_docs])
        prompt = f"""Context: {context}\n\nQuestion: {query}\n\nAnswer: """

        # Ensure the prompt is long enough for a full response
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

        # Initialize a list to store generated tokens
        generated_tokens = []

        # Generate response token by token
        output = self.model.generate(
            **inputs,
            max_length=max_length,
            num_beams=5,
            temperature=0.7,
            top_p=0.9,
            no_repeat_ngram_size=2,
            early_stopping=True,
            output_scores=True,
            return_dict_in_generate=True,
            # do_sample=True,
          num_return_sequences=1  # Reduce the number of return sequences


        )

        for token_id in output.sequences[0]:
            # Decode the token and add to the generated tokens list
            token = self.tokenizer.decode([token_id], skip_special_tokens=True)
            generated_tokens.append(token)

            # Check if both opening and closing single quotes are present in the generated output
            if "'" in generated_tokens and "'" in ''.join(generated_tokens):
                break

        # Join the tokens to form the final response
        response = ''.join(generated_tokens)
        return response


In [3]:
# export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [4]:
pip install chromadb

In [8]:
model_name = "facebook/rag-sequence-base"

from datasets import load_dataset

# The instruction dataset to use

dataset_name = "teksingh/NagarGPT-dataset0"

from datasets import Dataset

# Load dataset (you can process it here)

dataset = load_dataset(dataset_name, split="train")


In [4]:
import torch
torch.cuda.empty_cache()


In [10]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [ ]:
def main():

    # Initialize RAG system

    rag = RAGSystem()



    # Example documents

    documents = load_hf_dataset(dataset_name)



    # Add documents to system

    rag.add_documents(documents)



    # Example query

    # query = "जन्मदर्ता गर्नको लागी आवश्यक कागजातहरु के के चाहिन्छ"
    query = "मृत्युदर्ताको सेवा शुल्क तथा दस्तुर रकम के कति लाग्छ"

    response = rag.generate_response(query)



    print("Query:", query)

    #print("\nResponse:", textwrap.fill(response, width=80))
    print('Response',response)


if __name__ == "__main__":

    main()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
# accelerate config



In [ ]:
# pip install faiss-gpu


In [ ]:
import faiss

In [ ]:
# # Load tokenizer, retriever, and model
# from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration



# tokenizer = RagTokenizer.from_pretrained(model_name)



# retriever = RagRetriever.from_pretrained(

#     model_name,

#     index_name="exact",  # Uses exact retrieval (built on FAISS index)

#     passages_path=None  # None if using preloaded passages, or provide your dataset path

# )



# model = RagSequenceForGeneration.from_pretrained(model_name, retriever=retriever)





# # Set custom passages as the knowledge base for the retriever

# retriever.index_custom_dataset(dataset, "text")



In [ ]:
# # Example query

# query = "What is the capital of France?"



# # Tokenize the input query

# input_ids = tokenizer(query, return_tensors="pt").input_ids



# # Generate response

# output_ids = model.generate(input_ids)



# # Decode the output to text

# generated_text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]

# print("Generated Answer:", generated_text)
